In [18]:
import xarray as xr

In [2]:
atmos = xr.open_zarr("gs://vcm-ml-intermediate/2024-11-15-CM4-piControl-atmosphere-1deg-8layer-200yr.zarr")
atmos.time

<xarray.DataArray 'time' (time: 292000)> Size: 2MB
array([cftime.DatetimeNoLeap(151, 1, 1, 6, 0, 0, 0, has_year_zero=True),
       cftime.DatetimeNoLeap(151, 1, 1, 12, 0, 0, 0, has_year_zero=True),
       cftime.DatetimeNoLeap(151, 1, 1, 18, 0, 0, 0, has_year_zero=True), ...,
       cftime.DatetimeNoLeap(350, 12, 31, 12, 0, 0, 0, has_year_zero=True),
       cftime.DatetimeNoLeap(350, 12, 31, 18, 0, 0, 0, has_year_zero=True),
       cftime.DatetimeNoLeap(351, 1, 1, 0, 0, 0, 0, has_year_zero=True)],
      dtype=object)
Coordinates:
  * time     (time) object 2MB 0151-01-01 06:00:00 ... 0351-01-01 00:00:00
Attributes:
    axis:           T
    bounds:         time_bnds
    calendar_type:  NOLEAP
    long_name:      time

In [3]:
var_resampled = [
"DLWRFsfc",
"DSWRFsfc",
"ULWRFsfc",
"USWRFsfc",
"LHTFLsfc",
"SHTFLsfc",
"PRATEsfc",
"eastward_surface_wind_stress",
"northward_surface_wind_stress",
"surface_temperature",
"ocean_fraction",
"sea_ice_fraction",
]

In [4]:
atmos_subset = atmos[var_resampled]

In [5]:
atmos_5d = atmos_subset.resample(time="5D").mean()
atmos_5d.time

<xarray.DataArray 'time' (time: 14601)> Size: 117kB
array([cftime.DatetimeNoLeap(151, 1, 1, 0, 0, 0, 0, has_year_zero=True),
       cftime.DatetimeNoLeap(151, 1, 6, 0, 0, 0, 0, has_year_zero=True),
       cftime.DatetimeNoLeap(151, 1, 11, 0, 0, 0, 0, has_year_zero=True), ...,
       cftime.DatetimeNoLeap(350, 12, 22, 0, 0, 0, 0, has_year_zero=True),
       cftime.DatetimeNoLeap(350, 12, 27, 0, 0, 0, 0, has_year_zero=True),
       cftime.DatetimeNoLeap(351, 1, 1, 0, 0, 0, 0, has_year_zero=True)],
      dtype=object)
Coordinates:
  * time     (time) object 117kB 0151-01-01 00:00:00 ... 0351-01-01 00:00:00
Attributes:
    axis:           T
    bounds:         time_bnds
    calendar_type:  NOLEAP
    long_name:      time

In [7]:
for name in atmos.variables:
    if "ak" in name:
        atmos_5d[name] = atmos[name]
    if "bk" in name:
        atmos_5d[name] = atmos[name]

In [8]:
atmos_5d = atmos_5d.isel(time=slice(1, len(atmos_5d.time)))

In [9]:
chunks_config = {"time": 10, "lat": 180, "lon": 360}
atmos_5d = atmos_5d.chunk(chunks_config)

In [10]:
atmos_5d.surface_temperature.isel(time=slice(0,10), lat=90, lon=0).values

array([300.9013 , 300.9613 , 300.90747, 301.49258, 301.81836, 302.16852,
       302.5473 , 302.96036, 302.89243, 302.68103], dtype=float32)

In [13]:
output_store = "gs://vcm-ml-intermediate/2025-01-21-cm4-piControl-atmosphere-5day-sfc-vars/data-fixed.zarr"

In [14]:
print(f"Output dataset size is {atmos_5d.nbytes / 1e9} GB")

Output dataset size is 45.411961264 GB


In [17]:
atmos_5d.to_zarr(output_store)